In [3]:
import pickle
import os
import logging
import pandas as pd
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
from gensim.models import Word2Vec
import multiprocessing
from time import time
from indic_transliteration import detect
from indic_transliteration import sanscript
from indic_transliteration.detect import Scheme
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate

INFO - 22:09:32: NumExpr defaulting to 8 threads.


In [2]:
#creating a dictionary linking the start date of each week with a list containing the dates contained within
#each week

In [18]:
start_date = pd.to_datetime("2020-01-22", format = "%Y-%m-%d")
week_starts = [start_date + pd.Timedelta(i, "W") for i in range(19)]
week_days = {}
for week in week_starts:
    days = []
    if week_starts.index(week) == 18:
        for i in range(4):
            days.append(str((week + pd.Timedelta(i, "D")).date()))
        week_days[str(week.date())] = days
    else:
        for i in range(7):
            days.append(str((week + pd.Timedelta(i, "D")).date()))
        week_days[str(week.date())] = days
start_of_weeks = [str(week.date()) for week in week_starts]

In [19]:
week_starts

[Timestamp('2020-01-22 00:00:00'),
 Timestamp('2020-01-29 00:00:00'),
 Timestamp('2020-02-05 00:00:00'),
 Timestamp('2020-02-12 00:00:00'),
 Timestamp('2020-02-19 00:00:00'),
 Timestamp('2020-02-26 00:00:00'),
 Timestamp('2020-03-04 00:00:00'),
 Timestamp('2020-03-11 00:00:00'),
 Timestamp('2020-03-18 00:00:00'),
 Timestamp('2020-03-25 00:00:00'),
 Timestamp('2020-04-01 00:00:00'),
 Timestamp('2020-04-08 00:00:00'),
 Timestamp('2020-04-15 00:00:00'),
 Timestamp('2020-04-22 00:00:00'),
 Timestamp('2020-04-29 00:00:00'),
 Timestamp('2020-05-06 00:00:00'),
 Timestamp('2020-05-13 00:00:00'),
 Timestamp('2020-05-20 00:00:00'),
 Timestamp('2020-05-27 00:00:00')]

In [20]:
week_days

{'2020-01-22': ['2020-01-22',
  '2020-01-23',
  '2020-01-24',
  '2020-01-25',
  '2020-01-26',
  '2020-01-27',
  '2020-01-28'],
 '2020-01-29': ['2020-01-29',
  '2020-01-30',
  '2020-01-31',
  '2020-02-01',
  '2020-02-02',
  '2020-02-03',
  '2020-02-04'],
 '2020-02-05': ['2020-02-05',
  '2020-02-06',
  '2020-02-07',
  '2020-02-08',
  '2020-02-09',
  '2020-02-10',
  '2020-02-11'],
 '2020-02-12': ['2020-02-12',
  '2020-02-13',
  '2020-02-14',
  '2020-02-15',
  '2020-02-16',
  '2020-02-17',
  '2020-02-18'],
 '2020-02-19': ['2020-02-19',
  '2020-02-20',
  '2020-02-21',
  '2020-02-22',
  '2020-02-23',
  '2020-02-24',
  '2020-02-25'],
 '2020-02-26': ['2020-02-26',
  '2020-02-27',
  '2020-02-28',
  '2020-02-29',
  '2020-03-01',
  '2020-03-02',
  '2020-03-03'],
 '2020-03-04': ['2020-03-04',
  '2020-03-05',
  '2020-03-06',
  '2020-03-07',
  '2020-03-08',
  '2020-03-09',
  '2020-03-10'],
 '2020-03-11': ['2020-03-11',
  '2020-03-12',
  '2020-03-13',
  '2020-03-14',
  '2020-03-15',
  '2020-03-16',
 

In [21]:
start_of_weeks

['2020-01-22',
 '2020-01-29',
 '2020-02-05',
 '2020-02-12',
 '2020-02-19',
 '2020-02-26',
 '2020-03-04',
 '2020-03-11',
 '2020-03-18',
 '2020-03-25',
 '2020-04-01',
 '2020-04-08',
 '2020-04-15',
 '2020-04-22',
 '2020-04-29',
 '2020-05-06',
 '2020-05-13',
 '2020-05-20',
 '2020-05-27']

In [10]:
#creating a function that detects the language scheme of every word in a tweet. 
#if the scheme is not one of ("itrans", "hk", "slp1", "iast", "velthuis", "kolkata_v2" - these are all 
#Devanagari script to Roman script transliteration schemes), then the word is transliterated into Roman script
#using the Harvard-Kyoto("hk") scheme

In [11]:
def transliterate_hindi(word):
    lang = detect.detect(str(word))
    if lang not in [Scheme.itrans, Scheme.hk, Scheme.slp1, Scheme.iast, Scheme.velthuis, Scheme.kolkata_v2]:
        word = transliterate(word, lang, sanscript.HK).lower()
    else:
        word = word
    return word

In [15]:
text = "क्या kar rahe ho"

In [16]:
text = [transliterate_hindi(word) for word in text.lower().split()]
text

['kya', 'kar', 'rahe', 'ho']

In [17]:
#going through every week_start key in the week_days dictionary created above
#going through every day in the list associated with the week_start key
#looping through every state in for each day of the previous step and loading the file associated with that date
#transliterating every tweet and then saving all the tweets in a txt file which is common to that week
#saving the txt files in the "weekly training text for word2vec" folder

In [26]:
state_list = os.listdir("Cleaned Data")
t = time()
for start_ in start_of_weeks:
    t_1 = time()
    for day in week_days[start_]:
        t_2 = time()
        try:
            for state in state_list:
                _path = os.path.join("Cleaned Data", state)
                df = pickle.load(open(os.path.join(_path, "cleaned {}.csv.pkl".format(day)), "rb")) 
                for i in range(df.shape[0]):
                    tweet = df.at[i, "cleaned_tweet"].lower().split()
                    tweet = [transliterate_hindi(word) for word in tweet]
                    with open(os.path.join("weekly training text for word2vec\\{} week tweets.txt".format(start_)), "a", encoding = "utf-8") as f:
                        f.write(" ".join(tweet))
                        f.write("\n")
                        f.close()
            print("{} completed".format(day))
            print("time taken: {} mins".format(round((time() - t_2)/60, 2)))
        except:
            pass
    print("week starting {} completed".format(start_))
    print("time taken: {} mins".format(round((time() - t_1)/60, 2)))
print("time taken: {} mins".format(round((time() - t)/60, 2)))

2020-01-22 completed
time taken: 0.0 mins
2020-01-23 completed
time taken: 0.0 mins
2020-01-24 completed
time taken: 0.0 mins
2020-01-25 completed
time taken: 0.0 mins
2020-01-26 completed
time taken: 0.0 mins
2020-01-27 completed
time taken: 0.0 mins
2020-01-28 completed
time taken: 0.0 mins
week starting 2020-01-22 completed
time taken: 0.02 mins
2020-01-29 completed
time taken: 0.0 mins
2020-01-30 completed
time taken: 0.0 mins
2020-01-31 completed
time taken: 0.0 mins
2020-02-01 completed
time taken: 0.0 mins
2020-02-02 completed
time taken: 0.0 mins
2020-02-03 completed
time taken: 0.0 mins
2020-02-04 completed
time taken: 0.0 mins
week starting 2020-01-29 completed
time taken: 0.02 mins
2020-02-05 completed
time taken: 0.0 mins
2020-02-06 completed
time taken: 0.0 mins
2020-02-07 completed
time taken: 0.0 mins
2020-02-08 completed
time taken: 0.0 mins
2020-02-09 completed
time taken: 0.0 mins
2020-02-10 completed
time taken: 0.0 mins
2020-02-11 completed
time taken: 0.0 mins
week

In [ ]:
#if you are going through the entire thesis workflow, you must also save these txt files in a folder named the
#same within the "Contextual Analysis" main folder

In [28]:
#creating a function that will iterate over the weekly text file, reading it line by line and feeding it into 
#word2vec model. allows us to not load the whole txt file in memory. 

In [29]:
class MyCorpus:
    
    def __init__(self, file):
        self.filename = file
        
    def __iter__(self):
        with open(self.filename, encoding = "utf-8") as f:
            for row in f:
                _row = row.lower().split()
                yield _row

In [35]:
#creating functions that will create bigrams and trigrams for that week's tweet corpus, once again by
#reading the file line by line. the bigrams are created on the original tweets, and the trigrams are created on
#the bigrammed tweets. we consider only those bigrams and trigrams that occur at least 5 times a week

In [31]:
from gensim.models.phrases import Phrases
from gensim.models.phrases import Phraser

In [32]:
class MyBigramCorpus:
    
    def __init__(self, file, bigram_mod):
        self.filename = file
        
    def __iter__(self):
        with open(self.filename, encoding = "utf-8") as f:
            for row in f:
                _row = row.lower().split()
                yield bigram_mod[_row]

In [33]:
class MyTrigramCorpus:
    
    def __init__(self, file, bigram_mod, trigram_mod):
        self.filename = file
        
    def __iter__(self):
        with open(self.filename, encoding = "utf-8") as f:
            for row in f:
                _row = row.lower().split()
                yield trigram_mod[bigram_mod[_row]]

In [37]:
#iterating over each of these weekly txt files and training a separate word2vec model for each week
#saving these word2vec models in a folder called "wv models". the word2vec models are trained with 
#window size 5, vector size 100, CBOW architecture, and heirarchical softmax

In [40]:
t = time()
cores = multiprocessing.cpu_count()
for start_ in start_of_weeks:
    t_1 = time()
    sentences = MyCorpus("weekly training text for word2vec\\{} week tweets.txt".format(start_))
    bigrams = Phrases(sentences, min_count = 5, threshold = 7, delimiter = " ")
    bigram_mod = Phraser(bigrams)
    bigram_sentences = MyBigramCorpus("weekly training text for word2vec\\{} week tweets.txt".format(start_), bigram_mod)
    trigrams = Phrases(bigram_sentences, min_count = 5, threshold = 7, delimiter = " ")
    trigram_mod = Phraser(trigrams)
    trigram_sentences = MyTrigramCorpus("weekly training text for word2vec\\{} week tweets.txt".format(start_), bigram_mod, trigram_mod)
    w2v_model = Word2Vec(
        min_count = 20,
        window = 5,
        vector_size = 100,
        sample = 1e-5,
        alpha = 0.025,
        min_alpha = 0.0001,
        hs = 1,
        sg = 0,
        workers = cores - 2)
    w2v_model.build_vocab(trigram_sentences, progress_per = 10000)
    w2v_model.train(trigram_sentences, total_examples = w2v_model.corpus_count, epochs = 50, report_delay = 1)
    with open("weekly word2vec models\\week_{}.pkl".format(start_), "wb") as f:
        pickle.dump(w2v_model, f)
    print("week starting {} completed".format(start_))
    print("time taken: {} mins".format(round((time() - t_1)/60, 2)))
print("time taken: {} mins".format(round((time() - t)/60, 2)))

INFO - 23:07:54: collecting all words and their counts
INFO - 23:07:54: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 23:07:54: PROGRESS: at sentence #10000, processed 132288 words and 61618 word types
INFO - 23:07:54: collected 61618 token types (unigram + bigrams) from a corpus of 157742 words and 11832 sentences
INFO - 23:07:54: merged Phrases<61618 vocab, min_count=5, threshold=7, max_vocab_size=40000000>
INFO - 23:07:54: Phrases lifecycle event {'msg': 'built Phrases<61618 vocab, min_count=5, threshold=7, max_vocab_size=40000000> in 0.26s', 'datetime': '2023-04-01T23:07:54.531252', 'gensim': '4.2.0', 'python': '3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}
INFO - 23:07:54: exporting phrases from Phrases<61618 vocab, min_count=5, threshold=7, max_vocab_size=40000000>
INFO - 23:07:54: FrozenPhrases lifecycle event {'msg': 'exported FrozenPhrases<889 phrases, min_count=5, thresh

INFO - 23:08:07: EPOCH 25: training on 143334 raw words (8583 effective words) took 0.5s, 18949 effective words/s
INFO - 23:08:08: EPOCH 26: training on 143334 raw words (8556 effective words) took 0.5s, 17007 effective words/s
INFO - 23:08:08: EPOCH 27: training on 143334 raw words (8496 effective words) took 0.4s, 19041 effective words/s
INFO - 23:08:09: EPOCH 28: training on 143334 raw words (8475 effective words) took 0.4s, 19498 effective words/s
INFO - 23:08:09: EPOCH 29: training on 143334 raw words (8443 effective words) took 0.4s, 19621 effective words/s
INFO - 23:08:10: EPOCH 30: training on 143334 raw words (8598 effective words) took 0.4s, 19744 effective words/s
INFO - 23:08:10: EPOCH 31: training on 143334 raw words (8511 effective words) took 0.4s, 19224 effective words/s
INFO - 23:08:10: EPOCH 32: training on 143334 raw words (8478 effective words) took 0.4s, 19955 effective words/s
INFO - 23:08:11: EPOCH 33: training on 143334 raw words (8539 effective words) took 0.4s

week starting 2020-01-22 completed
time taken: 0.4 mins


INFO - 23:08:18: collected 56206 token types (unigram + bigrams) from a corpus of 140230 words and 10584 sentences
INFO - 23:08:18: merged Phrases<56206 vocab, min_count=5, threshold=7, max_vocab_size=40000000>
INFO - 23:08:18: Phrases lifecycle event {'msg': 'built Phrases<56206 vocab, min_count=5, threshold=7, max_vocab_size=40000000> in 0.25s', 'datetime': '2023-04-01T23:08:18.661486', 'gensim': '4.2.0', 'python': '3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}
INFO - 23:08:18: exporting phrases from Phrases<56206 vocab, min_count=5, threshold=7, max_vocab_size=40000000>
INFO - 23:08:18: FrozenPhrases lifecycle event {'msg': 'exported FrozenPhrases<695 phrases, min_count=5, threshold=7> from Phrases<56206 vocab, min_count=5, threshold=7, max_vocab_size=40000000> in 0.13s', 'datetime': '2023-04-01T23:08:18.790183', 'gensim': '4.2.0', 'python': '3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bi

INFO - 23:08:30: EPOCH 27: training on 129324 raw words (7321 effective words) took 0.4s, 19282 effective words/s
INFO - 23:08:31: EPOCH 28: training on 129324 raw words (7292 effective words) took 0.4s, 19361 effective words/s
INFO - 23:08:31: EPOCH 29: training on 129324 raw words (7479 effective words) took 0.4s, 19555 effective words/s
INFO - 23:08:32: EPOCH 30: training on 129324 raw words (7416 effective words) took 0.4s, 19268 effective words/s
INFO - 23:08:32: EPOCH 31: training on 129324 raw words (7419 effective words) took 0.4s, 19366 effective words/s
INFO - 23:08:32: EPOCH 32: training on 129324 raw words (7418 effective words) took 0.4s, 18857 effective words/s
INFO - 23:08:33: EPOCH 33: training on 129324 raw words (7323 effective words) took 0.4s, 18796 effective words/s
INFO - 23:08:33: EPOCH 34: training on 129324 raw words (7402 effective words) took 0.4s, 18515 effective words/s
INFO - 23:08:34: EPOCH 35: training on 129324 raw words (7345 effective words) took 0.4s

...

INFO - 23:18:35: EPOCH 24: training on 272474 raw words (25536 effective words) took 0.8s, 32865 effective words/s
INFO - 23:18:36: EPOCH 25: training on 272474 raw words (25075 effective words) took 0.8s, 32301 effective words/s
INFO - 23:18:37: EPOCH 26: training on 272474 raw words (25279 effective words) took 0.8s, 30056 effective words/s
INFO - 23:18:37: EPOCH 27: training on 272474 raw words (25207 effective words) took 0.8s, 31562 effective words/s
INFO - 23:18:38: EPOCH 28: training on 272474 raw words (25453 effective words) took 0.8s, 32166 effective words/s
INFO - 23:18:39: EPOCH 29: training on 272474 raw words (25331 effective words) took 0.8s, 31941 effective words/s
INFO - 23:18:40: EPOCH 30: training on 272474 raw words (25232 effective words) took 0.8s, 32439 effective words/s
INFO - 23:18:40: EPOCH 31: training on 272474 raw words (24949 effective words) took 0.8s, 32080 effective words/s
INFO - 23:18:41: EPOCH 32: training on 272474 raw words (25301 effective words) 

week starting 2020-05-20 completed
time taken: 0.72 mins


INFO - 23:18:56: collected 61917 token types (unigram + bigrams) from a corpus of 161632 words and 11076 sentences
INFO - 23:18:56: merged Phrases<61917 vocab, min_count=5, threshold=7, max_vocab_size=40000000>
INFO - 23:18:56: Phrases lifecycle event {'msg': 'built Phrases<61917 vocab, min_count=5, threshold=7, max_vocab_size=40000000> in 0.29s', 'datetime': '2023-04-01T23:18:56.038617', 'gensim': '4.2.0', 'python': '3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}
INFO - 23:18:56: exporting phrases from Phrases<61917 vocab, min_count=5, threshold=7, max_vocab_size=40000000>
INFO - 23:18:56: FrozenPhrases lifecycle event {'msg': 'exported FrozenPhrases<873 phrases, min_count=5, threshold=7> from Phrases<61917 vocab, min_count=5, threshold=7, max_vocab_size=40000000> in 0.12s', 'datetime': '2023-04-01T23:18:56.168270', 'gensim': '4.2.0', 'python': '3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bi

INFO - 23:19:09: EPOCH 27: training on 147554 raw words (9215 effective words) took 0.4s, 22217 effective words/s
INFO - 23:19:09: EPOCH 28: training on 147554 raw words (9017 effective words) took 0.5s, 19347 effective words/s
INFO - 23:19:10: EPOCH 29: training on 147554 raw words (9058 effective words) took 0.4s, 22150 effective words/s
INFO - 23:19:10: EPOCH 30: training on 147554 raw words (9363 effective words) took 0.4s, 22592 effective words/s
INFO - 23:19:10: EPOCH 31: training on 147554 raw words (9270 effective words) took 0.4s, 22624 effective words/s
INFO - 23:19:11: EPOCH 32: training on 147554 raw words (9189 effective words) took 0.4s, 21764 effective words/s
INFO - 23:19:11: EPOCH 33: training on 147554 raw words (9252 effective words) took 0.4s, 22260 effective words/s
INFO - 23:19:12: EPOCH 34: training on 147554 raw words (9075 effective words) took 0.4s, 22226 effective words/s
INFO - 23:19:12: EPOCH 35: training on 147554 raw words (9179 effective words) took 0.4s

week starting 2020-05-27 completed
time taken: 0.38 mins
time taken: 11.41 mins


In [ ]:
#if you are going through the entire thesis workflow, you must also save these word2vec models in a folder 
#named the same within the "Contextual Analysis" main folder